In [25]:
import sys, unittest, time, datetime
import urllib.request, urllib.error, urllib.parse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import InvalidArgumentException
from tqdm.notebook import tqdm

import pymongo
from pymongo.errors import DuplicateKeyError
from pymongo import MongoClient
import traceback
import pickle

In [5]:
client = MongoClient()
db = client.youtube
collection = db.popscience

# Получить список yuotube каналов

In [7]:
with open('channels_list.txt') as f:
    channels_list = f.read().splitlines()

In [8]:
len(channels_list), len(set(channels_list))

(391, 355)

In [9]:
channels_list = list(set(channels_list))

# Получение списка всех плейлистов 

In [16]:
%%time

yuotube_playlist_id_list = []

for channel_url in tqdm(channels_list): 
    driver = webdriver.Chrome('../chromedriver')
    driver.get(channel_url+"/playlists?view=1&sort=dd&shelf_id=0")
    time.sleep(5)
    height = driver.execute_script("return document.documentElement.scrollHeight")
    lastheight = 0

    while True:
        if lastheight == height:
            break
        lastheight = height
        driver.execute_script("window.scrollTo(0, " + str(height) + ");")
        time.sleep(2)
        height = driver.execute_script("return document.documentElement.scrollHeight")

    user_data = driver.find_elements_by_xpath('//*[@id="video-title"]')
    for i in user_data:
        url = i.get_attribute('href')
        playlist_id = url.split("=")[-1]
        yuotube_playlist_id_list.append(playlist_id)

    driver.close()
    driver.quit()

  0%|          | 0/355 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [17]:
yuotube_playlist_id_list

['PL2jwxGybEFiuQVQtrLPaH7GNB8ak29634',
 'PL2jwxGybEFity3pyl5SnPrgh7ERNCd4FD',
 'PL2jwxGybEFivRT-UOnvV0RAh1ykwwRrmc',
 'PL2jwxGybEFitFmGV0RdBNgbNBWFPtWRe0',
 'PLYI-NYBPZXmizTp-oucPyUJL0zSWkgADo',
 'PLYI-NYBPZXmjduCa-984TbdL_Tep3sH8G',
 'PLYI-NYBPZXmhcO7mj6ixBVy77tvfA8GFM',
 'PLYI-NYBPZXmhrm7Quz2x9tg8VsjXT3GQd',
 'PLYI-NYBPZXmhoQt0lOq7lil9qFxaFBAHY',
 'PLYI-NYBPZXmgieQfNXuu5JyinD3enTNct',
 'PLYI-NYBPZXmhGZFRoBnfimimeDRPYz5TT',
 'PLYI-NYBPZXmgIKFqU3d8kzy1pvtDZw52x',
 'PLYI-NYBPZXmglqnB0IHwtkrHH69EVvI-t']

In [18]:
with open("popscience_playlists.txt", 'w') as output:
    for row in yuotube_playlist_id_list:
        output.write(str(row) + '\n')